## Initialize the `SparkSession`

In [1]:
import getpass
import pyspark
from pyspark.sql import SparkSession

conf = pyspark.conf.SparkConf()
conf.setMaster('yarn')
conf.setAppName('final_project-{0}'.format(getpass.getuser()))
conf.set('spark.executor.memory', '6g')
conf.set('spark.executor.instances', '6')
conf.set('spark.port.maxRetries', '200')
sc = pyspark.SparkContext.getOrCreate(conf)
conf = sc.getConf()
sc

<SparkContext master=yarn appName=final_project-hlefebvr>

In [2]:
spark = SparkSession(sc)

In [3]:
from datetime import datetime

import pyspark.sql.functions as functions

### load whole dataset

In [4]:
whole_df = spark.read.csv("/datasets/project/istdaten/*/*/*.csv", header=True, sep=";")

- rename some useful columns

In [5]:
oldColumns = whole_df.schema.names
print(oldColumns)
newColumns = ["date", 'trip_id', 
              'BETREIBER_ID', 'BETREIBER_ABK',
              'BETREIBER_NAME', "transport_type", 
             "train_line", "train_service", 
              'UMLAUF_ID', 'VERKEHRSMITTEL_TEXT',
             "additional_trip", "failed_trip",
             'BPUIC', "station_name", "arrival_time",
             "actual_arrival", 'AN_PROGNOSE_STATUS',
             "departure_time", "actual_departure",
             'AB_PROGNOSE_STATUS', "DURCHFAHRT_TF"]

['BETRIEBSTAG', 'FAHRT_BEZEICHNER', 'BETREIBER_ID', 'BETREIBER_ABK', 'BETREIBER_NAME', 'PRODUKT_ID', 'LINIEN_ID', 'LINIEN_TEXT', 'UMLAUF_ID', 'VERKEHRSMITTEL_TEXT', 'ZUSATZFAHRT_TF', 'FAELLT_AUS_TF', 'BPUIC', 'HALTESTELLEN_NAME', 'ANKUNFTSZEIT', 'AN_PROGNOSE', 'AN_PROGNOSE_STATUS', 'ABFAHRTSZEIT', 'AB_PROGNOSE', 'AB_PROGNOSE_STATUS', 'DURCHFAHRT_TF']


In [6]:
whole_df = whole_df.toDF(*newColumns)
whole_df.printSchema()
# whole_df.show()

root
 |-- date: string (nullable = true)
 |-- trip_id: string (nullable = true)
 |-- BETREIBER_ID: string (nullable = true)
 |-- BETREIBER_ABK: string (nullable = true)
 |-- BETREIBER_NAME: string (nullable = true)
 |-- transport_type: string (nullable = true)
 |-- train_line: string (nullable = true)
 |-- train_service: string (nullable = true)
 |-- UMLAUF_ID: string (nullable = true)
 |-- VERKEHRSMITTEL_TEXT: string (nullable = true)
 |-- additional_trip: string (nullable = true)
 |-- failed_trip: string (nullable = true)
 |-- BPUIC: string (nullable = true)
 |-- station_name: string (nullable = true)
 |-- arrival_time: string (nullable = true)
 |-- actual_arrival: string (nullable = true)
 |-- AN_PROGNOSE_STATUS: string (nullable = true)
 |-- departure_time: string (nullable = true)
 |-- actual_departure: string (nullable = true)
 |-- AB_PROGNOSE_STATUS: string (nullable = true)
 |-- DURCHFAHRT_TF: string (nullable = true)



- dropping useless columns:

In [7]:
whole_df = whole_df.drop('BETREIBER_ID','BETREIBER_ABK', 'BETREIBER_NAME', 'UMLAUF_ID', 'BPUIC')

In [8]:
radius_stations_df = spark.read.csv('final_project/zurich_hb_stops.csv', header=True, sep=",")

In [9]:
filtered_df = whole_df.join(radius_stations_df, on="station_name", how='inner')

### THIRD TABLE: COMPUTING THE DELTAS DELAYS

- First assumptions: we filter entries were transport does not stop, entries that are additional trips and failed trips

In [10]:
third_table = filtered_df.filter((filtered_df.DURCHFAHRT_TF == False) \
                                 & (filtered_df.additional_trip == False)\
                                 & (filtered_df.failed_trip == False)).cache()

In [11]:
useful_trips = third_table.groupBy('trip_id').count().filter("count > 1")

In [12]:
useful_trips = useful_trips.drop('count')

- merging with our third table:

In [13]:
third_table = third_table.join(useful_trips, on="trip_id", how='inner').cache()

- if we just use only the GESCHAETZT entries, we will use only 4.8 % of our available data. Also, if we drop all the other entries there is the risk to lose information about some connection in our network. Right now we will just compute the difference of the columns. A further discussion with the other group members and TAs should be done.
- Is critical how to handle the null values, do we put delay = 0 if null? Or do we drop those entries? (by dropping them, we stil have the same problem highlighted in the first point of this cell. Right now I put a zero for all the null values.
- We also create a 'hour' column because it could be useful for the statitistics tests

In [14]:
from pyspark.sql.functions import when,unix_timestamp,hour,to_timestamp,col

In [15]:
third_table_final = third_table\
.withColumn('arrival_delay', when((third_table.actual_arrival.isNull()) \
                                  & (third_table.arrival_time.isNull()), None
                                 )
                                 .when((third_table.actual_arrival.isNull()) \
                                       & (third_table.arrival_time.isNotNull()), 0)\
                                 .otherwise(functions.round(unix_timestamp("actual_arrival",'dd.MM.yyyy HH:mm') - \
                                            unix_timestamp("arrival_time",'dd.MM.yyyy HH:mm')) / 60))\

.withColumn('departure_delay', when((third_table.actual_departure.isNull())\
                                    & (third_table.departure_time.isNull()), None
                                 )
                                 .when((third_table.actual_departure.isNull()) \
                                       & (third_table.departure_time.isNotNull()), 0)\
                                    .otherwise(functions.round(unix_timestamp("actual_departure",'dd.MM.yyyy HH:mm')\
                                               - unix_timestamp("departure_time",'dd.MM.yyyy HH:mm')) /60))\
.withColumn('hour',  when(third_table.arrival_time.isNull(), hour(to_timestamp(third_table.departure_time,
                                                                              'dd.MM.yyyy HH:mm'))) \
                         .otherwise(hour(to_timestamp(third_table.arrival_time, 'dd.MM.yyyy HH:mm'))))

### STATISTICS TETS

- In this section we will produce statistics tests in order to see if our bins delays can fit well a known distribution (we were thinking about a logNormal distr).
- First we will produce the test on the whole dataset, that means on the third table.
- then we will run a test on the delays on each station (group by station name)
- after this we will do the same for each transport line (group by the line_id, right now the columns is erroneusly called 'train_line')
- then, the same thing for each trip (group by trip_id)
- finally, same tests for hour of the day (we need to choose how to split the day, right now the column 'hour' has 24 distinct values of course).

In [16]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import lognorm
from pyspark.mllib.stat import Statistics
from scipy.stats import lognorm
from pyspark.sql.types import *

In [17]:
third_table_final.columns

['trip_id',
 'station_name',
 'date',
 'transport_type',
 'train_line',
 'train_service',
 'VERKEHRSMITTEL_TEXT',
 'additional_trip',
 'failed_trip',
 'arrival_time',
 'actual_arrival',
 'AN_PROGNOSE_STATUS',
 'departure_time',
 'actual_departure',
 'AB_PROGNOSE_STATUS',
 'DURCHFAHRT_TF',
 'id',
 'latitude',
 'longitude',
 'elevation',
 'dist_to_zurich_HB',
 'arrival_delay',
 'departure_delay',
 'hour']

Shall we consider that all delay distributions are log normals, no matter the groupby ? No function to guess the distrib, only tests to assess normality.

returns a tupple with two arrays, ticks and counts (pass number of bins into parameters, 20 here):

In [18]:
#temp=third_table_final.select('departure_delay').rdd.flatMap(lambda x: x).histogram(20)

In [19]:
#delay_per_station=third_table_final.groupBy('station_name','departure_delay').count().orderBy('station_name').collect()

In [20]:
#def plot_hist_delay(bins,arrival_or_departure):
    #temp=third_table_final.select(arrival_or_departure).rdd.flatMap(lambda x: x).histogram(bins)
   # plt.bar(temp)

In [21]:
#count_dep_delay_panda=third_table_final.groupBy('departure_delay').count().orderBy('departure_delay').toPandas()
#count_arr_delay_panda=third_table_final.groupBy('arrival_delay').count().orderBy('arrival_delay').toPandas()

We want to fit the lognormfit to all the delays per trip_id and line:

Windows:

In [22]:
#trip_window = Window.partitionBy('trip_id')
#train_line_window = Window.partitionBy('train_line')
#trip_stop_window=Window.partitionBy('trip_id','station_name')

Over variables  for arrival delays:

In [23]:
#dep_avg_tripid=avg("departure_delay").over(trip_window)
#dep_var_tripid=variance("departure_delay").over(trip_window)
#dep_avg_trainline=avg("departure_delay").over(train_line_window)
#dep_var_trainline=variance("departure_delay").over(train_line_window)

Over variables for departure delays:

In [24]:
#arr_avg_tripid=avg("arrival_delay").over(trip_window)
#arr_var_tripid=variance("arrival_delay").over(trip_window)
#arr_avg_trainline=avg("arrival_delay").over(train_line_window)
#arr_var_trainline=variance("arrival_delay").over(train_line_window)

Then we build train line and trip id dataset:

In [25]:
#tripid_avg_var = third_table_final.select('trip_id','departure_delay','arrival_delay',dep_avg_tripid.alias('average_dep_delay'),dep_var_tripid.alias('variance_dep_delay'),arr_avg_tripid.alias('average_arr_delay'),arr_var_tripid.alias('variance_arr_delay'))
#trainline_avg_var = third_table_final.select('train_line','departure_delay','arrival_delay',dep_avg_trainline.alias('average_dep_delay'),dep_var_trainline.alias('variance_dep_delay'),arr_avg_trainline.alias('average_arr_delay'),arr_var_trainline.alias('variance_arr_delay'))

Now we modify the dataset and gather for each trip_id and each train_line the delays as a list:

In [26]:
#df_train_line = trainline_avg_var.groupBy('train_line','average_dep_delay','variance_dep_delay','average_arr_delay','variance_arr_delay').agg(functions.collect_list("departure_delay").alias('list_dep_delay'),functions.collect_list("arrival_delay").alias('list_arr_delay'))

#df_trip_id=tripid_avg_var.groupBy('trip_id','average_dep_delay','variance_dep_delay','average_arr_delay','variance_arr_delay').agg(functions.collect_list("departure_delay").alias('list_dep_delay'),functions.collect_list("arrival_delay").alias('list_arr_delay'))


In [27]:
third_table_final_dep=third_table_final.where(col("departure_delay").isNotNull())
third_table_final_arr=third_table_final.where(col("arrival_delay").isNotNull())

In [ ]:
df_delay_dep = third_table_final_dep.groupBy('trip_id','station_name').agg(functions.collect_list("departure_delay").alias('list_dep_delay'))
df_delay_arr = third_table_final_arr.groupBy('trip_id','station_name').agg(functions.collect_list("arrival_delay").alias('list_arr_delay'))





In [ ]:
df_delay_dep_param=df_delay_dep.rdd.map(lambda x:[x['trip_id'], x['station_name'],x['list_dep_delay'],[float(b) for b in lognorm.fit(x['list_dep_delay'])]])
df_delay_dep_param=df_delay_arr.rdd.map(lambda x:[x['trip_id'], x['station_name'],x['list_arr_delay'],[float(b) for b in lognorm.fit(x['list_arr_delay'])]])


df_delay_dep_fit=df_delay_dep_param.map(lambda x:(x[0],x[1],x[2],x[3])).toDF(['trip_id','station_name','list_dep_delay','fit_param_dep'])
df_delay_arr_fit=df_delay_dep_param.map(lambda x:(x[0],x[1],x[2],x[3])).toDF(['trip_id','station_name','list_arr_delay','fit_param_arr'])

In [34]:
df_delay_arr_fit.show(2)

+---------------+-------------------+--------------------+--------------------+
|        trip_id|       station_name|      list_arr_delay|       fit_param_arr|
+---------------+-------------------+--------------------+--------------------+
|85:11:13752:001|     Birmensdorf ZH|[2.0, 1.0, -1.0, ...|[0.64744702021595...|
|85:11:13752:001|Bonstetten-Wettswil|[3.0, 3.0, 0.0, 1...|[0.64971286678057...|
+---------------+-------------------+--------------------+--------------------+
only showing top 2 rows



We split the parameters in 3 columns :

In [ ]:
df_delay_dep_param_split=df_delay_dep_fit.select('trip_id','station_name',df_delay_dep_fit["fit_param_dep"].getItem(0).alias("shape_dep"),df_delay_dep_fit["fit_param_dep"].getItem(1).alias("mean_dep"),df_delay_dep_fit["fit_param_dep"].getItem(2).alias("std_dep")).cache()
df_delay_arr_param_split=df_delay_arr_fit.select('trip_id','station_name',df_delay_arr_fit["fit_param_arr"].getItem(0).alias("shape_arr"),df_delay_arr_fit["fit_param_arr"].getItem(1).alias("mean_arr"),df_delay_arr_fit["fit_param_arr"].getItem(2).alias("std_arr")).cache()

In [36]:
df_delay_dep_param_split.show(2)

+---------------+-------------------+------------------+-------------------+------------------+
|        trip_id|       station_name|         shape_dep|           mean_dep|           std_dep|
+---------------+-------------------+------------------+-------------------+------------------+
|85:11:13752:001|     Birmensdorf ZH|0.6474470202159504|-1.6494151038426064|2.5714673710314537|
|85:11:13752:001|Bonstetten-Wettswil|0.6497128667805727|-0.7439320429616096|   2.7395542606923|
+---------------+-------------------+------------------+-------------------+------------------+
only showing top 2 rows



In [26]:
#df3bis=df_delay_dep_param_split.select([col(c).cast("string") for c in df_delay_dep_param_split.columns])

In [ ]:
#df3bis.write.csv('departure_fit_delays.csv')

In [ ]:
df_join=df_delay_dep_param_split.join(df_delay_arr_param_split,on=['trip_id','station_name'],how='inner').cache()


In [29]:
df_join.columns

['trip_id',
 'station_name',
 'shape_dep',
 'mean_dep',
 'std_dep',
 'shape_arr',
 'mean_arr',
 'std_arr']

In [ ]:
dict_param= {}

In [ ]:
for row in df_join:
    if row['trip_id'] in dict_param:
        continue
    dict_param[row['trip_id']]={}

In [ ]:
for id_trip in dict_param:
    df=df_join.filter(df_join['trip_id']==id_trip)
    for row in df:
        dict_param[id_trip][row['station_name']]={}

In [ ]:
for id_trip in dict_param:
    for stop in dict_param[id_trip]:
            df=df_join.filter((df_join['trip_id']==id_trip)&(df_join['station_name']=stop)
            dict_param[id_trip][stop]['shape_dep']=df.select('shape_dep').collect()
            dict_param[id_trip][stop]['mean_dep']=df.select('mean_dep').collect()
            dict_param[id_trip][stop]['std_dep']=df.select('std_dep').collect()
            dict_param[id_trip][stop]['shape_arr']=df.select('shape_arr').collect()
            dict_param[id_trip][stop]['mean_arr']=df.select('mean_arr').collect()
            dict_param[id_trip][stop]['std_arr']=df.select('std_arr').collect()

In [ ]:
df_join.rdd.saveAsPickleFile('parameters_fit')

###  Map probabilities function : 3 solutions - use RDD

First solution UDF

In [44]:
def proba_values(w,x,y,z):
    return lognorm.pdf(np.linspace(min(w),max(w),100),x,y,z)

In [45]:
udf_proba= functions.udf(proba_values, ArrayType(FloatType()))

In [46]:
df=df_delay_dep_param_split.withColumn("probabilities",udf_proba(df_delay_dep_param_split.list_dep_delay,df_delay_dep_param_split.shape,df_delay_dep_param_split.mean,df_delay_dep_param_split.std))

In [ ]:
df.show(2)

Second solution apply function directly

In [ ]:
df=df_delay_dep_param_split.withColumn("probabilities",lognorm.pdf(np.linspace(min(df_delay_dep_param_split['list_dep_delay']),max(df_delay_dep_param_split['list_dep_delay']),100),df_delay_dep_param_split['shape'

In [ ]:
df=df_delay_dep_param_split.withColumn("probabilities",lognorm.pdf(np.linspace(min(df_delay_dep_param_split['list_dep_delay']),max(df_delay_dep_param_split['list_dep_delay']),100),df_delay_dep_param_split['shape'],df_delay_dep_param_split['mean'],df_delay_dep_param_split['std']))


In [ ]:
df.show(2)

### Third solution use rdd: Good solution

In [65]:
df_rdd_dep=df_delay_dep_param_split.rdd.map(lambda x:[x['trip_id'], x['station_name'],x['shape'],x['mean'],x['std'],[float(b) for b in lognorm.pdf(np.linspace(min(x['list_dep_delay']),max(x['list_dep_delay']),100),x['shape'],x['mean'],x['std'])]])
df_rdd_arr=df_delay_arr_param_split.rdd.map(lambda x:[x['trip_id'], x['station_name'],x['shape'],x['mean'],x['std'],[float(b) for b in lognorm.pdf(np.linspace(min(x['list_arr_delay']),max(x['list_arr_delay']),100),x['shape'],x['mean'],x['std'])]])




In [ ]:
df_final_dep=df_rdd_dep.map(lambda x:(x[0],x[1],x[2],x[3],x[4],x[5])).toDF(['trip_id','station_name','shape','mean','std','probabilities'])
df_final_arr=df_rdd_arr.map(lambda x:(x[0],x[1],x[2],x[3],x[4],x[5])).toDF(['trip_id','station_name','shape','mean','std','probabilities'])


## CSV Export

In [26]:
from pyspark.sql.functions import col

df3bis=df3.fillna(0).select([col(c).cast("string") for c in df3.columns])

In [ ]:
df3bis.write.csv('parameters_lognorm_fit4.csv')

In [ ]:
df3pandas=df3.fillna(0).toPandas()

### END : CSV Export

## EXAMPLE :

In [ ]:
#example 
import numpy as np
import scipy.stats

# generate log-normal distributed set of samples
samples   = np.random.lognormal( mean=1., sigma=.4, size=10000 )

# make a fit to the samples and generate the resulting PDF
shape, loc, scale = scipy.stats.lognorm.fit( samples, floc=0 )
x_fit       = np.linspace( samples.min(), samples.max(), 100 )
samples_fit = scipy.stats.lognorm.pdf( x_fit, shape, loc=loc, scale=scale )

In [ ]:
samples_fit#output of lognom.pdf is an array

## END EXAMPLE

To compute proba such as : t+w<s, we need the cumulative distribution function:

In [ ]:
cdf_fitted=stats.lognorm.cdf(x_fit,scatter,loc,mean)

To find P[x<=Y]:

In [ ]:
scipy.stats.lognorm(mean, std).pdf(Y)

To find P[x>Y]:

In [ ]:
scipy.stats.lognorm(mean, std).sf(Y)

In [ ]:
sc.stop()